In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/predict

[Errno 2] No such file or directory: 'drive/MyDrive/predict'
/content/drive/MyDrive/predict


In [ ]:
!pip install pydub
!pip install spleeter 
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gc
def generate_image(scr_wav, dest_wav):
      file_vocals  = scr_wav
      y, sr = librosa.load(file_vocals)
      D = np.abs(librosa.stft(y))**2
      S = librosa.feature.melspectrogram(S=D)
      S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=512,
                                  fmax=8000)
      plt.figure(figsize=(10, 4))
      librosa.display.specshow(librosa.power_to_db(S,
                                                  ref=np.max),
                              y_axis=None, fmax=8000,
                            x_axis=None)

      print(file_vocals)
      plt.axis = None
      plt.savefig(dest_wav,bbox_inches ="tight",dpi = 100 ,pad_inches = 0 )
      # Giải phóng bộ nhớ của biến y và sr
      del y, sr
      # Thu gom bộ nhớ
      gc.collect()

      del S
      # Thu gom bộ nhớ
      gc.collect()

      plt.figure().clear()
      plt.close()
      plt.cla()
      plt.clf()
      

In [ ]:
num_cut = 30000
import librosa
import librosa.display
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import shutil
from google.colab import files
import spleeter
import pydub
import os
import time
from pydub import AudioSegment
def load_image(file_wav):
  lst_img = []
  audio = AudioSegment.from_file(file_wav, format="wav")
  if(len(audio)>num_cut):

    n = len(audio)//num_cut
    try:
      os.mkdir("/aucutter")
      os.mkdir("/spectogram")
    except:
      pass
    for i in range(0,n):
      cuter = audio[i*num_cut:(i+1)*num_cut]
      file_wav = "/aucutter/vocals"+str(i)+".wav"
      file_img = "/spectogram/img"+str(i)+".png"
      cuter.export(file_wav, format="wav")
      generate_image(file_wav,file_img)
      print("counting")
      img = cv2.resize(cv2.imread(file_img),(258,102))
      lst_img.append(img)
      os.remove(file_wav)
      os.remove(file_img)
    del audio
  else:
    y, sr = librosa.load(file_wav)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,
                                fmax=8000)
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.power_to_db(S,
                                                ref=np.max),
                            y_axis=None, fmax=8000,
                          x_axis=None)
    plt.axis = None
    plt.savefig("test.png",bbox_inches ="tight",dpi = 100 ,pad_inches = 0 )
    plt.tight_layout()
    img = cv2.resize(cv2.imread("test.png"),(258,102))
    lst_img.append(img)
  return np.array(lst_img)


In [ ]:
from spleeter import separator
name = os.listdir("data")[0]
file_name = "data"+"/"+name
separator = separator.Separator('spleeter:2stems')
separator.separate_to_file(file_name, "out")
file_wav = "out"+"/"+name.split(".")[0]+"/"+"vocals.wav"
lst_img = load_image(file_wav)
for km in os.listdir("out"):
  shutil.rmtree("out"+"/"+km)
plt.figure().clear()
plt.close()
plt.cla()
plt.clf()


INFO:spleeter:File out/BienTinhDuongNgocThai-6r7/vocals.wav written succesfully
INFO:spleeter:File out/BienTinhDuongNgocThai-6r7/accompaniment.wav written succesfully


INFO:spleeter:File out/BienTinhDuongNgocThai-6r7/accompaniment.wav written succesfully
INFO:spleeter:File out/BienTinhDuongNgocThai-6r7/vocals.wav written succesfully


/aucutter/vocals0.wav
counting
/aucutter/vocals1.wav
counting
/aucutter/vocals2.wav
counting
/aucutter/vocals3.wav
counting
/aucutter/vocals4.wav
counting
/aucutter/vocals5.wav
counting
/aucutter/vocals6.wav
counting
/aucutter/vocals7.wav
counting
/aucutter/vocals8.wav
counting
/aucutter/vocals9.wav


In [40]:
list_singer = ['CẨM LY','DAN_NGUYEN','DƯƠNG_NGOC_THAI','MINH TUYẾT','MỸ TÂM','QUANG_LE']
from tensorflow import keras
model = keras.models.load_model("model_ 0.780.h5")
pre = model.predict(np.array(lst_img))
pre = np.sum(pre, axis = 0)
c = np.round(pre,3)
print(c)
k = np.argmax(pre)
print(k)
max = np.max(pre)*100
list_singer = ['CẨM LY','DAN_NGUYEN','DƯƠNG_NGOC_THAI','MINH TUYẾT','MỸ TÂM','QUANG_LE']
# cẩm ly đứng thứ 1 trong list
if(k == 2):
  print("đây là: DƯƠNG NGỌC THÁI "+list_singer[k])
else:
  print("đây là 1 ca sĩ nào khác")


1/1 [==============================] - 1s 500ms/step
[2.400e-02 3.200e-02 9.154e+00 6.350e-01 1.510e-01 4.000e-03]
2
đây là: DƯƠNG NGỌC THAI DƯƠNG_NGOC_THAI
